This is phase 1 of the Smart Allocation Tool. The minimum viable product for this phase is to realize pallet ranking based on the following requirements:
1. Zespri orders
2. Market Restrictions
3. Storage Characteristics
4. LOP

In [32]:
import pandas as pd
import numpy as np
import pyodbc
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

In [33]:
# Set up parameters & SQL connection
uid = '_svc_jupyter'
pwd = 'happy!1'
driver = "{SQL Server}"
server_ods = "datastore,50903"
server_sql4="seeka-sql4"
database_ods = "ODS;"  
database_sql4 = "FreshSupplyStage;"

conn_ods = pyodbc.connect('DRIVER=' + driver + ';SERVER=' + server_ods + '' + 
                      ';DATABASE=' + database_ods + ';UID=' + uid + ';PWD=' + pwd)     

conn_sql4= pyodbc.connect('DRIVER=' + driver + ';SERVER=' + server_sql4 + '' + 
                      ';DATABASE=' + database_sql4 + ';UID=' + uid + ';PWD=' + pwd) 

In [40]:
#user input for order number
while True:
    try:
        OrderNumber=input("Please enter the Zespri Order Number: ")
        query_OE_count=f"SELECT COUNT(*) FROM [FreshSupplyStage].[dbo].[vw_Stage_OrderExtract] WHERE DeliveryRequestNo={OrderNumber}"
        cursor_OE=conn_sql4.cursor()
        cursor_OE.execute(query_OE_count)
        OE_rowcount=cursor_OE.fetchall()
        if OE_rowcount[0][0]>0:
            cursor_OE.execute(f"SELECT COUNT(*) FROM [FreshSupplyStage].[dbo].[vw_Stage_OrderExtract] WHERE DeliveryRequestNo={OrderNumber} AND LoadEndDate>GETDATE()")
            OE_rowcount_validdate=cursor_OE.fetchall()
            if OE_rowcount_validdate[0][0]>0:
                print('Successs')
                break
            else:
                print('Order Shipped already')
                
        else:
            print('does not exist')
    except ValueError:
        print('Invalid input')
            
            

Please enter the Zespri Order Number: 6463703
Successs


In [41]:
#user input to indicate kiwistart or not
while True:
    try:
        kiwistart=input(f"Is order {OrderNumber} Kiwistart? (Y/N):").strip().upper()
        if kiwistart in ['Y','N']:
            print(f"{OrderNumber}: Kiwstart Status {kiwistart}")
            break
        else:
            print("Invalid input")
    except ValueError:
        print('Invalid input')

Is order 6463703 Kiwistart? (Y/N):y
6463703: Kiwstart Status Y


In [42]:
#get the order line detail
query_line="SELECT Coolstore,DeliveryRequestLineNo,Country,VCGM,Size,PCAT,LabelIndr,CustomerLabel,QtyPallet,Market"+\
" FROM [FreshSupplyStage].[dbo].[vw_Stage_OrderExtract]"+\
f"WHERE DeliveryRequestNo={OrderNumber}"

cursor_OE.execute(query_line)

OrderLine=cursor_OE.fetchall()


In [43]:
#display order info:
print(f"Order Number: {OrderNumber} \n")
print(f"VarCGM: {OrderLine[0][3]} \n")
print(f"Country: {OrderLine[0][2]} \n")
print('Allocated Coolstores:',', '.join({item[0] for item in OrderLine})) #distinct sites allocated
print('----------------------------------------------')
for i in range(len(OrderLine)):
    print(f"OrderLine: {OrderLine[i][1]}")
    print(f"\t Site: {OrderLine[i][0]}")
    print(f"\t Size: {OrderLine[i][4]}")
    print(f"\t PackCategory: {OrderLine[i][5]}")
    print(f"\t LabelIndicator: {OrderLine[i][6]}")
    print(f"\t PalletOrdered: {OrderLine[i][8]}")
    print(f"\t CustomerLable: {OrderLine[i][7]}")


Order Number: 6463703 

VarCGM: GA1CK 

Country: CN 

Allocated Coolstores: 3AON, 5HUK, 3ENQ, 3XUU, 4OAK, 3USR, 3RNL, 3FGS
----------------------------------------------
OrderLine: 10
	 Site: 3AON
	 Size: 16
	 PackCategory: ENIJ
	 LabelIndicator: JB
	 PalletOrdered: 5
	 CustomerLable: C
OrderLine: 20
	 Site: 3AON
	 Size: 18
	 PackCategory: ENIJ
	 LabelIndicator: JB
	 PalletOrdered: 7
	 CustomerLable: C
OrderLine: 20
	 Site: 4OAK
	 Size: 18
	 PackCategory: ENIJ
	 LabelIndicator: JB
	 PalletOrdered: 8
	 CustomerLable: C
OrderLine: 30
	 Site: 4OAK
	 Size: 22
	 PackCategory: ENIJ
	 LabelIndicator: JB
	 PalletOrdered: 44
	 CustomerLable: C
OrderLine: 30
	 Site: 3ENQ
	 Size: 22
	 PackCategory: ENIJ
	 LabelIndicator: JB
	 PalletOrdered: 14
	 CustomerLable: C
OrderLine: 30
	 Site: 3FGS
	 Size: 22
	 PackCategory: ENIJ
	 LabelIndicator: JB
	 PalletOrdered: 8
	 CustomerLable: C
OrderLine: 30
	 Site: 3XUU
	 Size: 22
	 PackCategory: ENIJ
	 LabelIndicator: JB
	 PalletOrdered: 3
	 CustomerLable: C
Or

In [44]:
#############################################################################
#rule 1: if pallet is IT or IJ and has 2 or more holds on China, Japan and Taiwan, then should be prioritized
#rule 2: if the pallet has greater than or equal to 6 market holds, then should be proritized
#rule 3: rank by current LOP system
#rule 4: if it is GA and kiwistart, F>G>P; if it is HW and kiwistart, F>H; for RS 1>2>3>4>.....
    
#define the ranking function
def seeka_rank(row):
    #storage characteristic for Kiwistart
    if kiwistart=="Y":
        if OrderLine[j][3][:2]=="GA":
            if row['storechar']=="F":
                weight_ks=3
            elif row['storechar']=='G':
                weight_ks=2
            elif row['storchar']=='P':
                weight_ks=1
            else:
                weight_ks=0
        elif OrderLine[j][3][:2]=="HW":
            if row['storechar']=="K":
                weight_ks=2
            elif row['storechar']=="H":
                weight_ks=1
            else:
                weight_ks=0
        else:
            weight_ks=0
    else:
        if OrderLine[j][3][:2]=="RS":
            weight_ks=-int(row['storechar'])*0.1
        else:
            weight_ks=0
        
    #assigning weight to each rule
    weight_rule1=0.6
    weight_rule2=0.3
    weight_rule3=0.1
        
    #calculate score for each rule
    #rule1:
    if row['packtype'][:2] in ['IT','IJ']:
        if row['markethold_CN']+row['markethold_TW']+row['markethold_JP']>=2:
            rule_score1=(row['markethold_CN']+row['markethold_TW']+row['markethold_JP'])*weight_rule1
        else:
            rule_score1=0
    else:
        rule1_score=0
        
    #rule2:
    if row.filter(like='markethold').sum()>=6:
        rule2_score=row.filter(like='markethold').sum()*weight_rule2
    else:
        rule2_score=0
        
    #rule3:
    rule3_score=-int(str(row['lo_priority'])[0])*weight_rule3
        
    #total
    total_score=rule1_score+rule2_score+rule3_score+weight_ks
    return total_score

In [45]:
for j in range(len(OrderLine)):
    print("-------------------------------------------------")
    print(f"Looking for pallets for Order {OrderNumber}, Line {OrderLine[j][1]} at site {OrderLine[j][0]}.......")
    print(f"\t Size: {OrderLine[j][4]}")
    print(f"\t PackCategory: {OrderLine[j][5]}")
    print(f"\t LabelIndicator: {OrderLine[j][6]}")
    print(f"\t CustomerLabelCode: {OrderLine[j][7]}")
    print(f"\t PalletOrdered: {OrderLine[j][8]}")
    TrialIndicator_query="AND TrialPackingIndicatorCode<>'A'" if OrderLine[j][2]=='CN' and OrderLine[j][3][-2:]=='CK' else ""
    markethold_query=f"AND markethold_{OrderLine[j][9]}=0"
    query_pallet = ("SELECT * FROM [ODS].[dmt].[PalletInventory] " 
                   "WHERE instorestatus = 1 AND " 
                   f"Season = 2024 AND PalletOK_Status = 'Y' AND PalletOrderStatus NOT IN ('Preallocated','Allocated') AND FullPallet=1 AND "
                   f"site = '{OrderLine[j][0]}' AND " 
                   f"size = '{OrderLine[j][4]}' AND " 
                   #f"markethold_{OrderLine[j][2]} = 0 AND " 
                   f"variety = '{OrderLine[j][3][:2]}' AND " 
                   f"protocol NOT IN ('X','A') AND " 
                   f"class = '{OrderLine[j][3][2]}' AND " 
                   f"growmethod = '{OrderLine[j][3][-2:]}' AND " 
                   f"labelindr = '{OrderLine[j][6]}' AND "
                   f"CustomerLabelCode='{OrderLine[j][7]}' AND "
                   f"SUBSTRING(packtype,1,4) = '{OrderLine[j][5]}'" 
                   f"{TrialIndicator_query}"
                   f"{markethold_query}"
                   )
    
    df_pallet=pd.read_sql(query_pallet,conn_ods)
    print(f"Available Pallets: {df_pallet.shape[0]}")
    if df_pallet.shape[0]==0:
        print(f"No pallets avaialble for {OrderNumber}, Line {OrderLine[j][1]} ")
        continue
    elif df_pallet.shape[0]<OrderLine[j][8]:
        print(f"Not enough available pallets. {OrderLine[j][8]-df_pallet.shape[0]} pallets short.")
    
    ##########################################################################################
    #calculate the rank and sort by rank
    df_pallet['Rank']=df_pallet.apply(seeka_rank,axis=1)
    df_pallet=df_pallet.sort_values(by='Rank',ascending=False)
    
    #for the pallets with the same rank, sort them by room and row
    df_pallet['LocationRoomCode'] = df_pallet['LocationRoomCode'].fillna('') 
    df_pallet['row'] = df_pallet['row'].fillna('')
    df_pallet = df_pallet.groupby('Rank', sort=False,group_keys=False).apply(lambda x: x.sort_values(by=['LocationRoomCode', 'row'])).reset_index(drop=True)
    
    #modify the dataset and columns names for display
    df_pallet['CountMarketHolds']=df_pallet.filter(like='markethold').sum(axis=1)
    df_pallet=df_pallet.reset_index(drop=True)
    df_pallet.rename(columns={'TrialPackingIndicatorCode':'TrialIndr',
                             'markethold_CN':'Hold_CN',
                             'markethold_TW':'Hold_TW',
                             'markethold_JP':"Hold_JP",
                             'lo_priority':'LOP',
                             'CountMarketHolds':'Holds',
                             'storechar':'strchr',
                             'LocationRoomCode':'Room',
                             'row':'Row',
                             'CustomerLabelCode':'CustLabel'
                             },inplace=True)
    df_result=df_pallet[['pallet','Hold_CN','Hold_TW','Hold_JP','LOP','Holds','strchr','TrialIndr','protocol','CustLabel','Room','Row','Rank']]
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    print(df_result.head(OrderLine[j][8]))
    print('')
    
    
    
    
    

-------------------------------------------------
Looking for pallets for Order 6463703, Line 10 at site 3AON.......
	 Size: 16
	 PackCategory: ENIJ
	 LabelIndicator: JB
	 CustomerLabelCode: C
	 PalletOrdered: 5
Available Pallets: 2
Not enough available pallets. 3 pallets short.
     pallet  Hold_CN  Hold_TW  Hold_JP  LOP  Holds strchr TrialIndr protocol CustLabel  Room Row  Rank
0  35506132        0        1        0   13     10      F         N        B         C  CS09  09   5.9
1  35505814        0        1        0   13     10      F         N        B         C  CS09  09   5.9

-------------------------------------------------
Looking for pallets for Order 6463703, Line 20 at site 3AON.......
	 Size: 18
	 PackCategory: ENIJ
	 LabelIndicator: JB
	 CustomerLabelCode: C
	 PalletOrdered: 7
Available Pallets: 4
Not enough available pallets. 3 pallets short.
     pallet  Hold_CN  Hold_TW  Hold_JP  LOP  Holds strchr TrialIndr protocol CustLabel  Room Row  Rank
0  35506385        0       

Available Pallets: 1
     pallet  Hold_CN  Hold_TW  Hold_JP  LOP  Holds strchr TrialIndr protocol CustLabel  Room Row  Rank
0  35806829        0        0        0   13      2      F         N        N         C  CS03  17   2.9

-------------------------------------------------
Looking for pallets for Order 6463703, Line 30 at site 5HUK.......
	 Size: 22
	 PackCategory: ENIJ
	 LabelIndicator: JB
	 CustomerLabelCode: C
	 PalletOrdered: 8
Available Pallets: 2
Not enough available pallets. 6 pallets short.
     pallet  Hold_CN  Hold_TW  Hold_JP  LOP  Holds strchr TrialIndr protocol CustLabel  Room Row  Rank
0  35006922        0        0        0   13      2      F         N        B         C  CS08  09   2.9
1  35006861        0        0        0   13      2      F         N        B         C  CS08  09   2.9

-------------------------------------------------
Looking for pallets for Order 6463703, Line 40 at site 5HUK.......
	 Size: 25
	 PackCategory: ENIT
	 LabelIndicator: GR
	 CustomerLa

Available Pallets: 0
No pallets avaialble for 6463703, Line 50 
-------------------------------------------------
Looking for pallets for Order 6463703, Line 50 at site 3RNL.......
	 Size: 27
	 PackCategory: ENIT
	 LabelIndicator: GR
	 CustomerLabelCode: C
	 PalletOrdered: 10
Available Pallets: 0
No pallets avaialble for 6463703, Line 50 
-------------------------------------------------
Looking for pallets for Order 6463703, Line 50 at site 3USR.......
	 Size: 27
	 PackCategory: ENIT
	 LabelIndicator: GR
	 CustomerLabelCode: C
	 PalletOrdered: 3
Available Pallets: 3
     pallet  Hold_CN  Hold_TW  Hold_JP  LOP  Holds strchr TrialIndr protocol CustLabel  Room Row  Rank
0  35806805        0        0        0   13      2      F         N        C         C  CS03  17   2.9
1  35806928        0        0        0   13      2      F         N        C         C  CS03  17   2.9
2  35806652        0        0        0   13      2      F         N        C         C  CS03  17   2.9

-------------

Available Pallets: 5
     pallet  Hold_CN  Hold_TW  Hold_JP  LOP  Holds strchr TrialIndr protocol CustLabel  Room Row  Rank
0  35614516        0        0        0   13      1      F         N        C         C  CS04  23   2.9
1  35614547        0        0        0   13      1      F         N        C         C  CS04  23   2.9

-------------------------------------------------
Looking for pallets for Order 6463703, Line 90 at site 3USR.......
	 Size: 22
	 PackCategory: ENML
	 LabelIndicator: JB
	 CustomerLabelCode: C
	 PalletOrdered: 4
Available Pallets: 4
     pallet  Hold_CN  Hold_TW  Hold_JP  LOP  Holds strchr TrialIndr protocol CustLabel  Room Row  Rank
0  35806584        0        0        0   13      2      F         N        N         C  CS03  16   2.9
1  35806867        0        0        0   13      2      F         N        N         C  CS03  16   2.9
2  35806959        0        0        0   13      2      F         N        N         C  CS03  16   2.9
3  35806775        0    